In [52]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
# 예외처리
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, TimeoutException, NoSuchWindowException 

import requests
import re
import time
import pandas as pd
from bs4 import BeautifulSoup

# 사용할 csv 불러오기
* 파일 path 각자에 맞게 설정해줘야함.

In [2]:
# 파일 path 설정
file1 = '../data/2023_지역별 관광지 검색순위.csv'
file2 = '../data/23년 기준 내국인 관심 관광지.csv'
file3 = '../data/구석구석dataset.csv'

In [3]:
# csv 파일 read

# 검색순위
네비게이션_raw = pd.read_csv(file1, encoding='cp949')
네비게이션_df = 네비게이션_raw.copy()

# 내국인 관심 관광지
관심관광지_raw = pd.read_csv(file2, encoding='cp949')
관심관광지_df = 관심관광지_raw.copy()

# 구석구석
구석구석_raw = pd.read_csv(file3, encoding='utf-8', index_col=0)
구석구석_df = 구석구석_raw.copy()

## 구석구석 DF 전처리
* 'location' 컬럼을 쪼개서 '시군구명' 컬럼 새로 만들어서 '구'만 따로 집어 넣음.

In [4]:
for i in range(구석구석_df.shape[0]):
    구석구석_df.loc[i, '시군구명'] = 구석구석_df.location[i].split(' ')[1]

# CSV merge하기
* merge 할 때, '관광지명'과 '시군구명'를 기준으로 함.

## 네비게이션 + 관심관광지

In [5]:
merge1_raw = pd.merge(네비게이션_df, 관심관광지_df, left_on=['시/군/구', '관광지명'], right_on=['시군구명', '관광지명']) # 23개..?
merge1_df = merge1_raw.copy()

In [6]:
# 필요없는 컬럼 삭제하기
merge1_df = merge1_df.drop(columns=['광역시/도', '시/군/구', '시군구명', '시도명'])

* '도로명주소' 컬럼
1. 겹치는 주소가 있었고 크롤링할 때 주소를 이용해서 크롤링 할 것이기 때문에 겹치는 주소를 전처리해줬음.
    * '서울 종로구 사직로 161-0' : 경복궁, 광화문
    * 광화문 주소를 '서울 종로구'만 남김
2. 주소 중에서 '서울 OO구'까지만 적힌 것이 있었음. 이런 것들은 '관광지명'으로 크롤링할 예정임.


In [7]:
# 광화문 주소 바꿔주기
merge1_df.loc[8, '도로명주소'] = '서울 종로구'

# 네이버 리뷰 크롤링 하기
* 기본 : 주소로 크롤링, 예외 : 관광지명으로 크롤링
* 함수

In [16]:
# 주소 사용해서 크롤링 시작하기
def using_addr(x):
    # 주소 가져오기
    location = x
    # 예외처리
    try:
        # 네이버 모바일 지도로 이동 (10초 대기)
        driver.get(f"https://m.search.naver.com/search.naver?sm=mtp_sly.hst&where=m&query={location}")
        time.sleep(10)
        # 첫 번째로 뜨는 지역 선택하기
        # 버튼 설정
        search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'P7gyV')))

    except TimeoutException:
        print('다시 시도해주세요.')
        spot = None
    
    except NoSuchWindowException:
        print('다시 시도해주세요.')
        spot = None

    else:
        # 클릭
        search_button.click()
        
        # 관광지명 추출
        # <span class="GHAhO">여의도 한강공원</span>
        spot = driver.find_element(By.CLASS_NAME, 'GHAhO').text

    return spot

In [9]:
# 관광지명 사용해서 크롤링 시작하기
def using_spot(x):
    # 관광지명 가져오기
    spot = x
    # 해당 관광지 찾기
    driver.get(f"https://m.search.naver.com/search.naver?sm=mtp_sly.hst&where=m&query={spot}")
    time.sleep(10)
    
    # 해당지역 선택하기
    loc_xpath = '//*[@id="_title"]/a'
    # <div id="_title" class="LylZZ"><a href="https://m.place.naver.com/attraction/13490935?entry=plt" role="button"><span class="GHAhO">서래마을</span><span class="lnJFt">체험마을</span></a></div>
    # 버튼 설정
    search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, loc_xpath)))
    # 클릭
    search_button.click()
    
    # 도로명주소 추출
    # <span class="LDgIH">서울 서초구 반포4동</span>
    location = driver.find_element(By.CLASS_NAME, 'LDgIH').text

    return location

In [20]:
# 방문자리뷰 확인하고 리스트로 저장하기
def make_list(spot, location, list1):
    # 방문자리뷰 찾기
    #app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm > div.dAsGb > span:nth-child(2)
    find_visitor_review = driver.find_elements(By.CLASS_NAME, 'PXMot')
    
    for i, review in enumerate(find_visitor_review):
        if '방문자리뷰' in review.text:
            print(f'{i+1}번째 : 방문자리뷰 찾음.')
            # 방문자리뷰 클릭
            visitor_xpath = f'//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[{i+1}]/a'
            search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, visitor_xpath)))
            search_button.click()
            break
        # # 별점 기능 끝났다고 해서 별점 선택하지 않고 돌림
        # elif '별점' in review.text:
        #     print(f'{i}번째 : 별점 찾음.')
        #     star = review.text
        else:
            print(f'{i+1}번째 : 방문자리뷰 X')
            
    # 원하는 리뷰목록있는지 확인하기
    # print(driver.find_element(By.CLASS_NAME, 'place_section_header').text)
    # 없으면
    if '이런 점이 좋았어요' not in driver.find_element(By.CLASS_NAME, 'place_section_header').text:
        list1.append({'관광지명' : spot, '도로명주소' : location, '리뷰내용': None , '리뷰수': None, '참여인원수' : None})
        print('리뷰 목록 없음.\n 종료')
        
    # 있으면
    else:
        for i in range(5):  # 최대 5번 클릭 시도
            try:
                search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'dP0sq')))
                search_button.click()
                print(f"{i+1}번째 버튼 클릭 성공")
                    
            except TimeoutException:
                print(f"{i+1}번째 버튼이 존재하지 않아 클릭하지 않고 넘어갑니다.")
                break  # 버튼이 더 이상 존재하지 않으면 반복 중단
        
        # 리뷰 개수 세기
        num = len(driver.find_elements(By.CLASS_NAME, 'MHaAm'))
            
        # 정보 추출
        # 리뷰 개수만큼 반복
        for n in range(0, num):                 
            # 리뷰 내용
            contents = driver.find_elements(By.CLASS_NAME, 't3JSf')[n].text
            # 선택한 수
            review_num = driver.find_elements(By.CLASS_NAME, 'CUoLy')[n].text.split('\n')[1]
            # 참여인원수 추출
            # <span class="mholG">263명 참여</span>
            reviewer = driver.find_element(By.CLASS_NAME, 'mholG').text.split()[0]
            # 리스트에 관광지명, 도로명주소, 리뷰내용과 리뷰수를 딕셔너리로 담기
            list1.append({'관광지명' : spot, '도로명주소' : location, '리뷰내용': contents , '리뷰수': review_num, '참여인원수' : reviewer})
        
    return list1

## 네비게이션 + 관심관광지 크롤링하기

In [21]:
# 리뷰 내용 모음 리스트
results = []

for i, addr in enumerate(merge1_df.도로명주소):
    # 주소가 '서울 OO구' 밖에 없는 경우 찾기
    list = addr.split()
    
    # 주소가 '구'까지 밖에 없는 경우 ==> 관광지명으로 리뷰 크롤링하기
    if len(list) <= 2:
        # 웹 드라이버 실행하고 최대화
        driver = webdriver.Chrome()
        driver.maximize_window()
        # 관광지명 DF에서 뽑아줌
        spot = merge1_df.loc[i, '관광지명']
        location = using_spot(spot)
        results = make_list(spot, location, results)
        # 창 닫기
        driver.quit()
        
        # 결과 출력
        print(f'===== {i+1}번째 =====')
        print(results, len(results))
        
    # 전체 주소가 있는 경우 ==> 주소로 리뷰 크롤링하기
    else:
        # 웹 드라이버 실행하고 최대화
        driver = webdriver.Chrome()
        driver.maximize_window()
        # 주소 변수에 넣어주기
        location = addr
        spot = using_addr(location)
        
        # 예외처리
        # 조건문 사용
        # 예외일 경우
        if spot is None:
            # 점검할 때 사용하기 위해 'N'으로 넣음
            results.append({'관광지명' : merge1_df.loc[i, '관광지명'], '도로명주소' : location, '리뷰내용' : None , '리뷰수' : None, '참여인원수' : None})
            print('창이 뜨지 않음.\n종료')
            # 창닫기
            driver.quit()
        # 예외가 아닐 경우
        else:
            results = make_list(spot, location, results)
            # 창닫기
            driver.quit()
        
        # 하나의 관광지에 대한 결과 출력
        print(f'===== {i+1}번째 =====')
        print(results, len(results))
            

1번째 : 방문자리뷰 X
2번째 : 방문자리뷰 찾음.
1번째 버튼 클릭 성공
2번째 버튼 클릭 성공
3번째 버튼 클릭 성공
4번째 버튼 클릭 성공
5번째 버튼이 존재하지 않아 클릭하지 않고 넘어갑니다.
===== 1번째 =====
[{'관광지명': '여의도 한강공원', '도로명주소': '서울 영등포구 여의동로 330-0', '리뷰내용': '"뷰가 좋아요"', '리뷰수': '221', '참여인원수': '265명'}, {'관광지명': '여의도 한강공원', '도로명주소': '서울 영등포구 여의동로 330-0', '리뷰내용': '"산책로가 잘 되어있어요"', '리뷰수': '132', '참여인원수': '265명'}, {'관광지명': '여의도 한강공원', '도로명주소': '서울 영등포구 여의동로 330-0', '리뷰내용': '"사진이 잘 나와요"', '리뷰수': '111', '참여인원수': '265명'}, {'관광지명': '여의도 한강공원', '도로명주소': '서울 영등포구 여의동로 330-0', '리뷰내용': '"방문객이 많아요"', '리뷰수': '79', '참여인원수': '265명'}, {'관광지명': '여의도 한강공원', '도로명주소': '서울 영등포구 여의동로 330-0', '리뷰내용': '"볼거리가 많아요"', '리뷰수': '75', '참여인원수': '265명'}, {'관광지명': '여의도 한강공원', '도로명주소': '서울 영등포구 여의동로 330-0', '리뷰내용': '"관리가 잘 되어있어요"', '리뷰수': '68', '참여인원수': '265명'}, {'관광지명': '여의도 한강공원', '도로명주소': '서울 영등포구 여의동로 330-0', '리뷰내용': '"편의시설이 잘 되어있어요"', '리뷰수': '47', '참여인원수': '265명'}, {'관광지명': '여의도 한강공원', '도로명주소': '서울 영등포구 여의동로 330-0', '리뷰내용': '"주차하기 편해요"', '리뷰수': '32', '참여인원수': '265명'}, {'관광지명': '여의도 한강

# 데이터 프레임 만들기
* 방문자리뷰가 있지만 크롤링 되지 않고 넘어가는 문제 발생했었음. DF 만들고 다시 점검한 후 다시 하는 방법으로 일단 시도함.

In [37]:
# 처음 크롤링 한 것으로 DF 만들기
merge1_review_raw = pd.DataFrame(results)
merge1_review_df = merge1_review_raw.copy()

In [38]:
print(merge1_review_df.shape)
merge1_review_df.head(1)

(335, 5)


,관광지명,도로명주소,리뷰내용,리뷰수,참여인원수
0,여의도 한강공원,서울 영등포구 여의동로 330-0,"""뷰가 좋아요""",221,265명


In [41]:
# 리뷰내용이 'N'인 것 추출
condition = merge1_review_df.리뷰내용.isnull()

# 다시 크롤링 할거라서 임시로 저장해둠.
again = merge1_review_df[condition].reset_index(drop=True)
print(again.shape)

# 리뷰내용이 'N'인 것만 제외하기
merge1_review_df = merge1_review_df[~condition].reset_index(drop=True)
print(merge1_review_df.shape)

(3, 5)
(332, 5)


# 다시 크롤링하기
* None 인 것 다시 크롤링 해서 점검하기

In [43]:
results = []

# 주소가 네이버 지도에서 추출되었기 때문에 주소로 크롤링하는 방법 사용함.
for i, addr in enumerate(again.도로명주소):
    # 웹 드라이버 실행하고 최대화
    driver = webdriver.Chrome()
    driver.maximize_window()
    # 주소 변수에 넣어주기
    location = addr
    spot = using_addr(location)
    # 예외처리
    # 조건문 사용
    # 예외일 경우
    if spot is None:
        # 점검할 때 사용하기 위해 None으로 넣음
        results.append({'관광지명' : merge1_df.loc[i, '관광지명'], '도로명주소' : location, '리뷰내용' : None , '리뷰수' : None, '참여인원수' : None})
        print('창이 뜨지 않음.\n종료')
        # 종료하기
        driver.quit()
    # 예외가 아닐 경우
    else:
        results = make_list(spot, location, results)
        # 창닫기
        driver.quit()
        
    # 하나의 관광지에 대한 결과 출력
    print(f'===== {i+1}번째 =====')
    print(results, len(results))

1번째 : 방문자리뷰 X
2번째 : 방문자리뷰 찾음.
1번째 버튼 클릭 성공
2번째 버튼 클릭 성공
3번째 버튼 클릭 성공
4번째 버튼이 존재하지 않아 클릭하지 않고 넘어갑니다.
===== 1번째 =====
[{'관광지명': '하늘공원', '도로명주소': '서울 마포구 하늘공원로 95-0', '리뷰내용': '"산책로가 잘 되어있어요"', '리뷰수': '394', '참여인원수': '681명'}, {'관광지명': '하늘공원', '도로명주소': '서울 마포구 하늘공원로 95-0', '리뷰내용': '"뷰가 좋아요"', '리뷰수': '323', '참여인원수': '681명'}, {'관광지명': '하늘공원', '도로명주소': '서울 마포구 하늘공원로 95-0', '리뷰내용': '"사진이 잘 나와요"', '리뷰수': '232', '참여인원수': '681명'}, {'관광지명': '하늘공원', '도로명주소': '서울 마포구 하늘공원로 95-0', '리뷰내용': '"볼거리가 많아요"', '리뷰수': '199', '참여인원수': '681명'}, {'관광지명': '하늘공원', '도로명주소': '서울 마포구 하늘공원로 95-0', '리뷰내용': '"피크닉하기 좋아요"', '리뷰수': '190', '참여인원수': '681명'}, {'관광지명': '하늘공원', '도로명주소': '서울 마포구 하늘공원로 95-0', '리뷰내용': '"편의시설이 잘 되어있어요"', '리뷰수': '59', '참여인원수': '681명'}, {'관광지명': '하늘공원', '도로명주소': '서울 마포구 하늘공원로 95-0', '리뷰내용': '"주차하기 편해요"', '리뷰수': '54', '참여인원수': '681명'}, {'관광지명': '하늘공원', '도로명주소': '서울 마포구 하늘공원로 95-0', '리뷰내용': '"대중교통이 편해요"', '리뷰수': '41', '참여인원수': '681명'}, {'관광지명': '하늘공원', '도로명주소': '서울 마포구 하늘공원로 95-0', '리뷰내용': '"가격이 합리적이에요"

# 다시 DF 만들기
* 점검한 내용을 원래의 DF에 추가해주기

In [45]:
# 다시 크롤링한 것 데이터프레임 만들기
add_review_raw = pd.DataFrame(results)
add_review_df = add_review_raw.copy()

,관광지명,도로명주소,리뷰내용,리뷰수,참여인원수
0,여의도 한강공원,서울 영등포구 여의동로 330-0,"""뷰가 좋아요""",221,265명
1,여의도 한강공원,서울 영등포구 여의동로 330-0,"""산책로가 잘 되어있어요""",132,265명
2,여의도 한강공원,서울 영등포구 여의동로 330-0,"""사진이 잘 나와요""",111,265명
3,여의도 한강공원,서울 영등포구 여의동로 330-0,"""방문객이 많아요""",79,265명
4,여의도 한강공원,서울 영등포구 여의동로 330-0,"""볼거리가 많아요""",75,265명
...,...,...,...,...,...
17,하늘공원,서울 마포구 하늘공원로 95-0,"""근처에 갈 곳이 많아요""",1,681명
18,하늘공원,서울 마포구 하늘공원로 95-0,"""신기한 식물이 많아요""",1,681명
19,하늘공원,서울 마포구 하늘공원로 95-0,"""경관이 독특해요""",1,681명
20,청와대,서울 종로구 청와대로 1-0,None,None,None


In [48]:
# 먼저 만들었던 DF와 합치기
merge1_review = pd.concat([merge1_review_df, add_review_df], axis=0).reset_index(drop=True)

In [51]:
merge1_review.head()

,관광지명,도로명주소,리뷰내용,리뷰수,참여인원수
0,여의도 한강공원,서울 영등포구 여의동로 330-0,"""뷰가 좋아요""",221,265명
1,여의도 한강공원,서울 영등포구 여의동로 330-0,"""산책로가 잘 되어있어요""",132,265명
2,여의도 한강공원,서울 영등포구 여의동로 330-0,"""사진이 잘 나와요""",111,265명
3,여의도 한강공원,서울 영등포구 여의동로 330-0,"""방문객이 많아요""",79,265명
4,여의도 한강공원,서울 영등포구 여의동로 330-0,"""볼거리가 많아요""",75,265명


In [49]:
# csv 저장하기
# merge1_review.to_csv('./검색순위_관심관광지_크롤링.csv')

In [50]:
# 확인
# pd.read_csv('./검색순위_관심관광지_크롤링.csv', encoding='utf-8', index_col=0)

,관광지명,도로명주소,리뷰내용,리뷰수,참여인원수
0,여의도 한강공원,서울 영등포구 여의동로 330-0,"""뷰가 좋아요""",221.0,265명
1,여의도 한강공원,서울 영등포구 여의동로 330-0,"""산책로가 잘 되어있어요""",132.0,265명
2,여의도 한강공원,서울 영등포구 여의동로 330-0,"""사진이 잘 나와요""",111.0,265명
3,여의도 한강공원,서울 영등포구 여의동로 330-0,"""방문객이 많아요""",79.0,265명
4,여의도 한강공원,서울 영등포구 여의동로 330-0,"""볼거리가 많아요""",75.0,265명
...,...,...,...,...,...
349,하늘공원,서울 마포구 하늘공원로 95-0,"""근처에 갈 곳이 많아요""",1.0,681명
350,하늘공원,서울 마포구 하늘공원로 95-0,"""신기한 식물이 많아요""",1.0,681명
351,하늘공원,서울 마포구 하늘공원로 95-0,"""경관이 독특해요""",1.0,681명
352,청와대,서울 종로구 청와대로 1-0,NaN,NaN,NaN
